# Final Project
| Group 1:       |     |
| -------------- | --- |
| Student Names:             |UCIDs:|
| Samiul Haque               |30169845|
| Elias Poitras-Whitecalf    |30193066|
| Ryan Graham                |30171130|



### Imports


In [ ]:
import kagglehub
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
import matplotlib
import warnings
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import sklearn.model_selection as sk
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KNN
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore') #ignoring some deprication warnings

path = kagglehub.dataset_download("evangora/premier-league-data")

print("Path to dataset files:", path)

Path to dataset files: /Users/eliaspw/.cache/kagglehub/datasets/evangora/premier-league-data/versions/1


### KNN


### Gradient Boosted Forest

### Linear Regression